<a href="https://colab.research.google.com/github/leito77/Sport_cars_clasifier/blob/main/merge_docking_GCN_valid_and_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **notebook created to analyze the enrichments coming from the work "Graphical neural networks and molecular docking as two complementary approaches for virtual projection: a case study on Cruzain"**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Folder to save
data_dir = '/content/drive/My Drive/augmented_gcn/combined_approach/'

In [ ]:
# read gcn predicions
dataset_GCN_preds = pd.read_csv(data_dir+"GCN_valid-test_preds_with_cids.txt")
dataset_GCN_preds.rename(columns={'PubChem CID':'PUBCHEM_CID'}, inplace=True)
dataset_GCN_preds.shape


(38800, 3)

In [ ]:
dataset_GCN_preds.head()

,preds,labels,PUBCHEM_CID
0,0.158217,1.0,5282219
1,0.178083,0.0,1459825
2,0.243947,0.0,802875
3,0.058142,0.0,2386010
4,0.773753,1.0,6603233


In [ ]:
# read in docking scores 
VS_table = pd.read_csv(data_dir+"ALL_BEST_report_all.csv") # Dockings without PFs
VS_table['PUBCHEM_COMPOUND_CID'] = list(map(lambda x: np.int(np.float(x)),VS_table['PUBCHEM_COMPOUND_CID']))
VS_table.rename(columns={'PUBCHEM_COMPOUND_CID':'PUBCHEM_CID'}, inplace=True)
VS_table.drop_duplicates(subset=['PUBCHEM_CID'], inplace=True)
print(VS_table.shape)
VS_table.head()

<ipython-input-6-ac332fadeef3>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  VS_table['PUBCHEM_COMPOUND_CID'] = list(map(lambda x: np.int(np.float(x)),VS_table['PUBCHEM_COMPOUND_CID']))
<ipython-input-6-ac332fadeef3>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/de

(73709, 6)


,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR
0,1,1727,1727,-13.558,-9.232,0.0
1,2,2532,2532,-17.640,-13.677,0.0
2,3,2540,2540,-19.179,-16.436,0.0
3,4,2603,2603,-17.421,-14.823,0.0
4,5,2993,2993,-15.270,-13.705,0.0


In [ ]:
# merge docking scores and predictions 
df_gcn = pd.merge(VS_table, dataset_GCN_preds, on='PUBCHEM_CID',how='inner')
docking_GCN_merged_sorted = VS_table.sort_values(by='SCORE.INTER')
docking_GCN_merged_sorted.head()
docking_GCN_merged_sorted.shape

(73709, 6)

In [ ]:
docking_GCN_merged_sorted.head()

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR
46341,45550,3247329,3247329,-30.672,-34.065,0.0
46330,45539,3247316,3247316,-35.398,-29.788,0.0
46317,45526,3247302,3247302,-31.776,-29.774,0.0
62364,61573,9625759,9625759,-39.269,-29.658,0.0
46342,45551,3247331,3247331,-28.126,-29.520,0.0


In [ ]:
# read in activity data from PubChem 
table = pd.read_csv(data_dir+"AID_1478_datatable.csv", skiprows=(1,2,3,4))
table = table[['PUBCHEM_CID','PUBCHEM_ACTIVITY_OUTCOME']] # keep only compound name and labels (i.e. Active, Inactive or Inconclusive)
table.dropna(inplace=True)
table['PUBCHEM_CID'] = list(map(lambda x: np.int(x),table['PUBCHEM_CID']))
table.drop_duplicates(subset=['PUBCHEM_CID'], inplace=True)
table.head()

<ipython-input-9-2e80b62536d2>:2: DtypeWarning: Columns (6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(data_dir+"AID_1478_datatable.csv", skiprows=(1,2,3,4))
<ipython-input-9-2e80b62536d2>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  table['PUBCHEM_CID'] = list(map(lambda x: np.int(x),table['PUBCHEM_CID']))


,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME
0,2532,Active
1,6604907,Active
2,6604717,Active
3,72430,Active
4,439487,Active


In [ ]:
# add activity data to the dataframe 
df = pd.merge(docking_GCN_merged_sorted, table, on='PUBCHEM_CID',how='inner')
df.head()

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,PUBCHEM_ACTIVITY_OUTCOME
0,45550,3247329,3247329,-30.672,-34.065,0.0,Inactive
1,45539,3247316,3247316,-35.398,-29.788,0.0,Inactive
2,45526,3247302,3247302,-31.776,-29.774,0.0,Inactive
3,61573,9625759,9625759,-39.269,-29.658,0.0,Inactive
4,45551,3247331,3247331,-28.126,-29.520,0.0,Inactive


In [ ]:
df.shape

(73709, 7)

In [ ]:
# add activity data to the dataframe 
df_gcn = pd.merge(df_gcn, table, on='PUBCHEM_CID',how='inner')
df_gcn.head()

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,preds,labels,PUBCHEM_ACTIVITY_OUTCOME
0,1,1727,1727,-13.558,-9.232,0.0,0.054910,1.0,Active
1,5,2993,2993,-15.270,-13.705,0.0,0.262217,1.0,Active
2,6,3117,3117,-16.278,-10.148,0.0,0.119474,1.0,Active
3,8,3565,3565,-17.977,-13.762,0.0,0.216423,1.0,Active
4,11,3799,3799,-16.134,-15.369,0.0,0.058563,1.0,Active


In [ ]:
df_gcn[df_gcn['preds'] >= 0.5].shape # Change preds value

(3149, 9)

In [ ]:
df_gcn.head()

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,preds,labels,PUBCHEM_ACTIVITY_OUTCOME
0,1,1727,1727,-13.558,-9.232,0.0,0.054910,1.0,Active
1,5,2993,2993,-15.270,-13.705,0.0,0.262217,1.0,Active
2,6,3117,3117,-16.278,-10.148,0.0,0.119474,1.0,Active
3,8,3565,3565,-17.977,-13.762,0.0,0.216423,1.0,Active
4,11,3799,3799,-16.134,-15.369,0.0,0.058563,1.0,Active


In [ ]:
# Plot ROC Curve based on docking scores only 
%matplotlib notebook
from sklearn import metrics
import matplotlib.pyplot as plt
y = np.array(df['labels'])
scores = np.array(-df['SCORE.INTER'])
fpr, tpr, threshold = metrics.roc_curve(y, scores, pos_label=1.0)
roc_auc= metrics.auc(fpr, tpr)
fig, ax = plt.subplots(figsize=(10,10))

ax.plot(fpr, tpr, lw=2, label='AUC = {:0.3f}'.format(roc_auc))
ax.set_xlabel('False Positive Rate', fontsize=16)
ax.set_ylabel('True Positive Rate', fontsize=16)
ax.legend(loc='lower right', fontsize=13, frameon=False)
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_aspect('equal')
plt.show()
#plt.savefig(data_dir+"ROC-curve.png") #to save
plt.close()

KeyError: ignored

def compute_roc_metrics(df, top,GCN=False):

    # Create slices of top and bottom docking-ranked compounds 
    top_slice = df.iloc[:top]
    bottom_slice = df.iloc[top:]

    # count active compounds that are predicted as active / inactive by 1) the GCN (tp_gcn / fn_gcn) and by docking (tp_dock / fn_dock) according to the current threshold 
    tp_gcn_tp_dock = top_slice[(top_slice['preds'] > 0) & (top_slice['labels'] > 0)].shape[0]
    fn_gcn_tp_dock = top_slice[(top_slice['preds'] < 1) & (top_slice['labels'] > 0)].shape[0]
    tp_gcn_fn_dock = bottom_slice[(bottom_slice['preds'] > 0) & (bottom_slice['labels'] > 0)].shape[0]
    fn_gcn_fn_dock = bottom_slice[(bottom_slice['preds'] < 1) & (bottom_slice['labels'] > 0)].shape[0]

    # count inactive compounds that are predicted as active / inactive by 1) the GCN (fp_gcn / tn_gcn) and by docking (fp_dock / tn_dock) according to the current threshold 
    fp_gcn_fp_dock = top_slice[(top_slice['preds'] > 0) & (top_slice['labels'] < 1)].shape[0]
    tn_gcn_fp_dock = top_slice[(top_slice['preds'] < 1) & (top_slice['labels'] < 1)].shape[0]
    fp_gcn_tn_dock = bottom_slice[(bottom_slice['preds'] > 0) & (bottom_slice['labels'] < 1)].shape[0]
    tn_gcn_tn_dock = bottom_slice[(bottom_slice['preds'] < 1) & (bottom_slice['labels'] < 1)].shape[0]

   
    # Comptute docking tp / fn / fp / tn regardless of GCN outcome 
    tp_dock = tp_gcn_tp_dock + fn_gcn_tp_dock
    fn_dock = tp_gcn_fn_dock + fn_gcn_fn_dock
    fp_dock = fp_gcn_fp_dock + tn_gcn_fp_dock
    tn_dock = fp_gcn_tn_dock + tn_gcn_tn_dock

    # Compute tpr / fpr for docking regardless of GCN outcome 
    tpr_dock = tp_dock / (tp_dock + fn_dock)
    fpr_dock = fp_dock / (fp_dock + tn_dock)
        
    total=tp_dock + fp_dock + fn_dock + tn_dock
    N = tp_dock + fp_dock
        
    #print("Docking:     TPR=",round(tpr_dock,3),"; FPR=", round(fpr_dock,3),"; N=", tp_dock + fp_dock,"; total=",tp_dock + fp_dock + fn_dock + tn_dock)
    return (tpr_dock, fpr_dock, N, total)        

In [ ]:
### NEW
def compute_roc_metrics(df, top,GCN=False):

    if GCN == True:  
      thr_value = 0.5

      # Create slices of top and bottom docking-ranked compounds 
      top_slice = df.iloc[:top]
      bottom_slice = df.iloc[top:]

      # count active compounds that are predicted as active / inactive by 1) the GCN (tp_gcn / fn_gcn) and by docking (tp_dock / fn_dock) according to the current threshold 
      tp_gcn_tp_dock = top_slice[(top_slice['preds'] > thr_value) & (top_slice['labels'] > 0)].shape[0]
      fn_gcn_tp_dock = top_slice[(top_slice['preds'] < thr_value) & (top_slice['labels'] > 0)].shape[0]
      tp_gcn_fn_dock = bottom_slice[(bottom_slice['preds'] > thr_value) & (bottom_slice['labels'] > 0)].shape[0]
      fn_gcn_fn_dock = bottom_slice[(bottom_slice['preds'] < thr_value) & (bottom_slice['labels'] > 0)].shape[0]

      # count inactive compounds that are predicted as active / inactive by 1) the GCN (fp_gcn / tn_gcn) and by docking (fp_dock / tn_dock) according to the current threshold 
      fp_gcn_fp_dock = top_slice[(top_slice['preds'] > thr_value) & (top_slice['labels'] < 1)].shape[0]
      tn_gcn_fp_dock = top_slice[(top_slice['preds'] < thr_value) & (top_slice['labels'] < 1)].shape[0]
      fp_gcn_tn_dock = bottom_slice[(bottom_slice['preds'] > thr_value) & (bottom_slice['labels'] < 1)].shape[0]
      tn_gcn_tn_dock = bottom_slice[(bottom_slice['preds'] < thr_value) & (bottom_slice['labels'] < 1)].shape[0]

   
      # Comptute docking tp / fn / fp / tn regardless of GCN outcome 
      tp_dock = tp_gcn_tp_dock + fn_gcn_tp_dock
      fn_dock = tp_gcn_fn_dock + fn_gcn_fn_dock
      fp_dock = fp_gcn_fp_dock + tn_gcn_fp_dock
      tn_dock = fp_gcn_tn_dock + tn_gcn_tn_dock

      # Compute tpr / fpr for docking regardless of GCN outcome 
      tpr_dock = tp_dock / (tp_dock + fn_dock)
      fpr_dock = fp_dock / (fp_dock + tn_dock)
        
      total=tp_dock + fp_dock + fn_dock + tn_dock
      N = tp_dock + fp_dock
        
      #print("Docking:     TPR=",round(tpr_dock,3),"; FPR=", round(fpr_dock,3),"; N=", tp_dock + fp_dock,"; total=",tp_dock + fp_dock + fn_dock + tn_dock)
      return (tpr_dock, fpr_dock, N, total) 
    
    else:
      # Create slices of top and bottom docking-ranked compounds 
      top_slice = df.iloc[:top]
      bottom_slice = df.iloc[top:]

      # count active compounds that are predicted as active / inactive by 1) the GCN (tp_gcn / fn_gcn) and by docking (tp_dock / fn_dock) according to the current threshold 
      tp_gcn_tp_dock = top_slice[(top_slice['preds'] > 0) & (top_slice['labels'] > 0)].shape[0]
      fn_gcn_tp_dock = top_slice[(top_slice['preds'] < 1) & (top_slice['labels'] > 0)].shape[0]
      tp_gcn_fn_dock = bottom_slice[(bottom_slice['preds'] > 0) & (bottom_slice['labels'] > 0)].shape[0]
      fn_gcn_fn_dock = bottom_slice[(bottom_slice['preds'] < 1) & (bottom_slice['labels'] > 0)].shape[0]

      # count inactive compounds that are predicted as active / inactive by 1) the GCN (fp_gcn / tn_gcn) and by docking (fp_dock / tn_dock) according to the current threshold 
      fp_gcn_fp_dock = top_slice[(top_slice['preds'] > 0) & (top_slice['labels'] < 1)].shape[0]
      tn_gcn_fp_dock = top_slice[(top_slice['preds'] < 1) & (top_slice['labels'] < 1)].shape[0]
      fp_gcn_tn_dock = bottom_slice[(bottom_slice['preds'] > 0) & (bottom_slice['labels'] < 1)].shape[0]
      tn_gcn_tn_dock = bottom_slice[(bottom_slice['preds'] < 1) & (bottom_slice['labels'] < 1)].shape[0]


      # Comptute docking tp / fn / fp / tn regardless of GCN outcome 
      tp_dock = tp_gcn_tp_dock + fn_gcn_tp_dock
      fn_dock = tp_gcn_fn_dock + fn_gcn_fn_dock
      fp_dock = fp_gcn_fp_dock + tn_gcn_fp_dock
      tn_dock = fp_gcn_tn_dock + tn_gcn_tn_dock

      # Compute tpr / fpr for docking regardless of GCN outcome 
      tpr_dock = tp_dock / (tp_dock + fn_dock)
      fpr_dock = fp_dock / (fp_dock + tn_dock)

      total=tp_dock + fp_dock + fn_dock + tn_dock
      N = tp_dock + fp_dock

      #print("Docking:     TPR=",round(tpr_dock,3),"; FPR=", round(fpr_dock,3),"; N=", tp_dock + fp_dock,"; total=",tp_dock + fp_dock + fn_dock + tn_dock)
      return (tpr_dock, fpr_dock, N, total)


### OLD Sort values according to gcn predictions (top-rank of preds = 1)
preds_active = df[df['preds'] > 0]

preds_inactive = df[df['preds'] < 1]

df_gcn = preds_active.append(preds_inactive)

In [ ]:
# merge docking scores and predictions 
#docking_GCN_merged = pd.merge(VS_table, dataset_GCN_preds, on='PUBCHEM_CID',how='inner')
docking_GCN_merged_sorted = VS_table.sort_values(by='SCORE.INTER')
docking_GCN_merged_sorted.head()
docking_GCN_merged_sorted.shape

(73709, 6)

In [ ]:
### NEW
# Sort values according to gcn predictions (top-rank of preds = 1)
preds_active = df_gcn[df_gcn['preds'] >= 0.5]
preds_inactive = df_gcn[df_gcn['preds'] < 0.5]
df_gcn = preds_active.append(preds_inactive)

<ipython-input-18-f2930e8030c5>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gcn = preds_active.append(preds_inactive)


In [ ]:
df_gcn = df_gcn.sort_values(by="preds", ascending=False)
df_gcn

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,preds,labels,PUBCHEM_ACTIVITY_OUTCOME
11771,57177,6399289,6399289,-17.960,-15.017,1.388,0.898365,0.0,Inactive
11896,57761,6604293,6604293,-25.021,-7.502,0.000,0.895749,0.0,Inactive
12503,60628,9555971,9555971,-16.191,-14.602,0.000,0.886688,0.0,Inactive
180,502,3856851,3856851,-25.805,-18.929,0.000,0.886666,1.0,Active
11907,57825,6604396,6604396,-20.697,-14.239,0.000,0.886413,0.0,Inactive
...,...,...,...,...,...,...,...,...,...
456,786,5558,5558,-11.701,-7.374,0.000,0.044154,0.0,Inactive
537,1237,7839,7839,-9.196,-4.820,0.000,0.044060,0.0,Inactive
483,926,6342,6342,-10.421,-6.089,0.000,0.043933,0.0,Inactive
308,63,702,702,-12.122,-7.374,0.000,0.043925,0.0,Inactive


In [ ]:
### Testing shape about docking and GCN
if df.shape[0] == df_gcn.shape[0]:
    print(True)
else:
    print(False)

False


In [ ]:
# Compute TPR and FPR for different thresholds 
def compute_thr(df, GCN):
    TPR = []
    FPR = []
    thr = []
    db_size = df.shape[0]
    n = int(db_size * 1 / 100)  # start with a size of 1% of the database 
    while n <= db_size: 
        val = compute_roc_metrics(df,n,GCN)
        TPR.append(val[0])
        FPR.append(val[1])
        thr.append(val[2])
        n += 100
    return TPR, FPR, thr

In [ ]:
TPR_dock, FPR_dock, thr_dock = compute_thr(df, GCN=False)
TPR_gcn, FPR_gcn, thr_gcn = compute_thr(df_gcn, GCN=True)

KeyError: ignored

In [ ]:
# Find index in TPR, FPR arrays threshold value (top X% of docking sorted database)
thr = int(df.shape[0] * 5 / 100)
idx_dock = np.where(np.array(thr_dock) >= thr)[0][1] 
idx_gcn = np.where(np.array(thr_gcn) >= thr)[0][1]

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt 
from sklearn import metrics
roc_auc_dock= metrics.auc(FPR_dock, TPR_dock)
roc_auc_gcn= metrics.auc(FPR_gcn, TPR_gcn)

fig, ax = plt.subplots(figsize=(6,6))
ax.plot(FPR_dock, TPR_dock,lw=2,label='docking (AUC = {:0.2f})'.format(roc_auc_dock))
ax.plot(FPR_gcn, TPR_gcn, lw=2,label='GCN + docking (AUC = {:0.2f})'.format(roc_auc_gcn))
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlabel('False Positive Rate', fontsize=16)
ax.set_ylabel('True Positive Rate', fontsize=16)
ax.legend(loc='lower right', fontsize=13, frameon=False)
#ax.plot([FPR_dock[idx_dock], FPR_gcn[idx_gcn]], [0, 1], color='gray', lw=1, linestyle='--')

#ax.plot(FPR_dock[idx_dock], TPR_dock[idx_dock], color='gray', marker='o', markersize=3)
#ax.plot(FPR_gcn[idx_gcn], TPR_gcn[idx_gcn], color='gray', marker='o', markersize=3)
#plt.savefig(data_dir+"GCN-ROC-merge_only_actives.png", dpi=600) #to save
plt.close()

# Enrichment análisis

## DOCKING

In [ ]:
df.head(15) # Contains all data from DOCKINGS without PFs and GCN predictions

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,PUBCHEM_ACTIVITY_OUTCOME
0,45550,3247329,3247329,-30.672,-34.065,0.000,Inactive
1,45539,3247316,3247316,-35.398,-29.788,0.000,Inactive
2,45526,3247302,3247302,-31.776,-29.774,0.000,Inactive
3,61573,9625759,9625759,-39.269,-29.658,0.000,Inactive
4,45551,3247331,3247331,-28.126,-29.520,0.000,Inactive
5,45524,3247299,3247299,-31.074,-29.444,0.000,Inactive
6,45525,3247300,3247300,-31.195,-29.168,0.000,Inactive
7,31906,2377732,2377732,-32.419,-28.878,0.000,Inactive
8,63985,15990289,15990289,-29.351,-28.633,0.000,Inactive
9,21311,1046866,1046866,-36.928,-28.413,0.000,Inactive


In [ ]:
%cd /content/drive/My Drive/augmented_gcn/combined_approach/

/content/drive/My Drive/augmented_gcn/combined_approach


In [ ]:
### Load Scoring module (from RDkit) and compute AUC (ROC) and Enrichment curve 
# Note that we have not load the RDkit packeage but only the Scoring.py which is in the current (local) working directory 
import Scoring 
y_docking = np.array(df['PUBCHEM_ACTIVITY_OUTCOME'])
scores_docking = np.array(-df['SCORE.INTER'])
y_asToF = (y_docking=='Active') #como verdadero o falso
scores_docking2 = np.array([scores_docking,y_asToF]).T
auc = Scoring.CalcAUC(scores_docking2, 1)
auc_docking = round(auc, 3)
print(auc_docking)

0.629


In [ ]:
#Compute enrichments 

fractions = np.arange(0.0005,1.0, 0.01)
#Scoring.CalcEnrichment(scores=scores2, col=1, fractions=fractions)

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
fractions = np.arange(0.0005,1.0, 0.01)
curve = Scoring.CalcPorc(scores_docking2, 1, fractions) #curve = [porcActives, porcTotal]


# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_docking,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve[1], curve[0])
plt.legend(['random', 'enrichment'])
plt.title('Docking priority enrichment')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

#plt.savefig(data_dir+"docking_priority_enrichment.png") #to save
plt.close()

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve = Scoring.CalcPorc(scores2, 1, fractions) #curve = [porcActives, porcTotal]


# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_docking,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve[1], curve[0])
plt.ylim(0, 40)
plt.xlim(0, 40)
plt.legend(['random', 'enrichment'])
plt.title('Docking priority enrichment')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

#plt.savefig(data_dir+"docking_priority_enrichment_zoom40-40.png") #to save
plt.close()

In [ ]:
# Get the molecules on the first 1% (then narrow down to 0.3%) of database 

import math
numMol = len(scores_docking)
numPerFrac = [math.ceil(numMol*f) for f in fractions]
onePerc = np.where(fractions >= .03)[0][0]
numMol_onePerc = int(numPerFrac[onePerc]) # number of molecules in the first 0.3 percent of database 
print(numMol_onePerc)

2249


In [ ]:
# Print out True Positives and False Positives from the top ranked 0.3% of database
df_onePerc = df.iloc[:numMol_onePerc,:]
TP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
FP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']

print("Number of True Positive samples {}".format(TP.shape[0]))
print("Number of False Positive samples {}".format(FP.shape[0]))
print("% Actives {}".format(float(TP.shape[0]) / float(FP.shape[0])))
#TP['PUBCHEM_CID'].to_csv('TPs.txt', index=False)
#FP['PUBCHEM_CID'].to_csv('FPs.txt', index=False)
Actives_docking = round((float(TP.shape[0]) / (float(FP.shape[0])+float(TP.shape[0])))*100,2)
print(Actives_docking)

Number of True Positive samples 61
Number of False Positive samples 2186
% Actives 0.027904849039341262
2.71


## DOCKING using PFs

In [ ]:
# Read in table with VS results 
VS_table_PFs = pd.read_csv(data_dir+"ALL_BEST_report_all-PF4.csv")
VS_table_PFs['PUBCHEM_COMPOUND_CID'] = list(map(lambda x: np.int(np.float(x)),VS_table_PFs['PUBCHEM_COMPOUND_CID']))
VS_table_PFs.rename(columns={'PUBCHEM_COMPOUND_CID':'PUBCHEM_CID'}, inplace=True)
VS_table_PFs.drop_duplicates(subset=['PUBCHEM_CID'], inplace=True)
print(VS_table_PFs.shape)
VS_table_PFs.head()

(51963, 6)


<ipython-input-28-6303f1f6d1f4>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  VS_table_PFs['PUBCHEM_COMPOUND_CID'] = list(map(lambda x: np.int(np.float(x)),VS_table_PFs['PUBCHEM_COMPOUND_CID']))
<ipython-input-28-6303f1f6d1f4>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://nu

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR
0,1,2532,2532,-17.858,-16.013,0.888
1,2,2540,2540,-12.965,-7.344,0.817
2,3,2603,2603,-8.103,-7.525,3.636
3,4,3503,3503,-22.219,-17.257,5.192
4,5,3565,3565,-1.937,-5.957,8.306


In [ ]:
# merge docking scores and predictions 
#docking_GCN_merged_PF4 = pd.merge(VS_table_PFs, dataset_GCN_preds, on='PUBCHEM_CID',how='inner')
docking_GCN_merged_PF4 = VS_table_PFs.sort_values(by='SCORE.INTER')

docking_GCN_merged_PF4.shape

(51963, 6)

In [ ]:
docking_GCN_merged_PF4.head()

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR
30968,30356,3247329,3247329,-27.008,-34.334,3.593
30947,30335,3247305,3247305,-25.483,-30.437,3.661
38811,38199,5832549,5832549,-33.149,-30.242,1.771
30942,30330,3247299,3247299,-28.775,-30.149,2.771
34252,33640,4270232,4270232,-27.636,-29.278,1.148


In [ ]:
VS_table_with_labels_PFs = pd.merge(docking_GCN_merged_PF4, table, on='PUBCHEM_CID',how='inner')
#VS_table_with_labels['SCORE.INTER.RESTR'] = 1 * VS_table_with_labels['SCORE.INTER'] + 1 * VS_table_with_labels['SCORE.RESTR']
#VS_table_with_labels['LE'] = VS_table_with_labels['SCORE.INTER'] / VS_table_with_labels['PUBCHEM_HEAVY_ATOM_COUNT']
VS_table_with_labels_PFs = VS_table_with_labels_PFs.sort_values(by='SCORE.INTER')
VS_table_with_labels_PFs.shape
#VS_table_with_labels[VS_table_with_labels['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active'].shape

(51963, 7)

In [ ]:
VS_table_with_labels_PFs.head()

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,PUBCHEM_ACTIVITY_OUTCOME
0,30356,3247329,3247329,-27.008,-34.334,3.593,Inactive
1,30335,3247305,3247305,-25.483,-30.437,3.661,Inactive
2,38199,5832549,5832549,-33.149,-30.242,1.771,Inactive
3,30330,3247299,3247299,-28.775,-30.149,2.771,Inactive
4,33640,4270232,4270232,-27.636,-29.278,1.148,Inactive


In [ ]:
### Load Scoring module (from RDkit) and compute AUC (ROC) and Enrichment curve 
# Note that we have not load the RDkit packeage but only the Scoring.py which is in the current (local) working directory 
#import rdkit
import Scoring 

y_PF4 = np.array(VS_table_with_labels_PFs['PUBCHEM_ACTIVITY_OUTCOME'])
scores_PF4 = np.array(-VS_table_with_labels_PFs['SCORE.INTER'])
y_asToF = (y_PF4=='Active') #como verdadero o falso
scores_PF4_2 = np.array([scores_PF4,y_asToF]).T
auc = Scoring.CalcAUC(scores_PF4_2, 1)
auc_PFs = round(auc, 3)
print(auc_PFs)

0.69


In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve = Scoring.CalcPorc(scores_PF4_2, 1, fractions) #curve = [porcActives, porcTotal]


# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_PF4,y_as_bin]).T # Change scores
curve_random = Scoring.CalcPorc(scores3, 1, fractions)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve[1], curve[0])
#plt.ylim(0, 40)
#plt.xlim(0, 40)
plt.legend(['random', 'enrichment'])
plt.title('Docking priority enrichment using FPs')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

plt.savefig(data_dir+"docking_FPs_priority_enrichment-2.png") #to save
plt.close()

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve = Scoring.CalcPorc(scores_PF4_2, 1, fractions) #curve = [porcActives, porcTotal]


# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_PF4,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve[1], curve[0])
plt.ylim(0, 40)
plt.xlim(0, 40)
plt.legend(['random', 'enrichment'])
plt.title('Docking priority enrichment using FPs')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

plt.savefig(data_dir+"docking_FPs_priority_enrichment-zoom40-40.png") #to save
plt.close()

In [ ]:
# Get the molecules on the first 1% (then narrow down to 0.3%) of database 
fractions = np.arange(0.0005,1.0, 0.01)
import math
numMol = len(scores_PF4_2)
numPerFrac = [math.ceil(numMol*f) for f in fractions]
onePerc = np.where(fractions >= .03)[0][0]
numMol_onePerc = int(numPerFrac[onePerc]) # number of molecules in the first 0.3 percent of database 
print(numMol_onePerc)

In [ ]:
# Print out True Positives and False Positives from the top ranked 0.3% of database
df_onePerc = VS_table_with_labels_PFs.iloc[:numMol_onePerc,:]
TP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
FP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']

print("Number of True Positive samples {}".format(TP.shape[0]))
print("Number of False Positive samples {}".format(FP.shape[0]))
print("% Actives {}".format(float(TP.shape[0]) / float(FP.shape[0])))
TP['PUBCHEM_CID'].to_csv('TPs.txt', index=False)
FP['PUBCHEM_CID'].to_csv('FPs.txt', index=False)
Actives_PF4 = round((float(TP.shape[0]) / (float(FP.shape[0])+float(TP.shape[0])))*100,2)
print(Actives_PF4)

## GCN then DOCKING

In [ ]:
df_gcn

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,preds,labels,PUBCHEM_ACTIVITY_OUTCOME
11771,57177,6399289,6399289,-17.960,-15.017,1.388,0.898365,0.0,Inactive
11896,57761,6604293,6604293,-25.021,-7.502,0.000,0.895749,0.0,Inactive
12503,60628,9555971,9555971,-16.191,-14.602,0.000,0.886688,0.0,Inactive
180,502,3856851,3856851,-25.805,-18.929,0.000,0.886666,1.0,Active
11907,57825,6604396,6604396,-20.697,-14.239,0.000,0.886413,0.0,Inactive
...,...,...,...,...,...,...,...,...,...
456,786,5558,5558,-11.701,-7.374,0.000,0.044154,0.0,Inactive
537,1237,7839,7839,-9.196,-4.820,0.000,0.044060,0.0,Inactive
483,926,6342,6342,-10.421,-6.089,0.000,0.043933,0.0,Inactive
308,63,702,702,-12.122,-7.374,0.000,0.043925,0.0,Inactive


In [ ]:
### Load Scoring module (from RDkit) and compute AUC (ROC) and Enrichment curve 
# Note that we have not load the RDkit packeage but only the Scoring.py which is in the current (local) working directory 
import Scoring 
y_GCN = np.array(df_gcn['PUBCHEM_ACTIVITY_OUTCOME'])
scores_GCN = np.array(-df_gcn['preds'])
y_asToF = (y_GCN=='Active') #como verdadero o falso
scores_GCN_2 = np.array([scores_GCN,y_asToF]).T
auc = Scoring.CalcAUC(scores_GCN_2, 1)
auc_GCN = round(auc, 3)
print(auc_GCN)

0.806


In [ ]:
#Compute enrichments 

#fractions = np.arange(0.0005,1.0, 0.01)
#Scoring.CalcEnrichment(scores=scores2, col=1, fractions=fractions)

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve = Scoring.CalcPorc(scores_GCN_2, 1, fractions) #curve = [porcActives, porcTotal]


# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_GCN,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve[1], curve[0])
plt.legend(['random', 'enrichment'])
plt.title('GCN filter + Docking priority enrichment')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

#plt.savefig(data_dir+"GCN_docking_merge_enrichment.png") #to save
plt.close()

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve = Scoring.CalcPorc(scores_GCN_2, 1, fractions) #curve = [porcActives, porcTotal]


# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_GCN,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve[1], curve[0])
plt.ylim(0, 40)
plt.xlim(0, 40)
plt.legend(['random', 'enrichment'])
plt.title('GCN filter + Docking priority enrichment')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

#plt.savefig(data_dir+"GCN_docking_merge_enrichment_zoom40-40.png") #to save
plt.close()

In [ ]:
# Get the molecules on the first 1% (then narrow down to 0.3%) of database 

import math
numMol = len(scores)
numPerFrac = [math.ceil(numMol*f) for f in fractions]
onePerc = np.where(fractions >= .03)[0][0]
numMol_onePerc = int(numPerFrac[onePerc]) # number of molecules in the first 0.3 percent of database 
numMol_onePerc

457

In [ ]:
# Print out True Positives and False Positives from the top ranked 0.3% of database
df_onePerc = df_gcn.iloc[:numMol_onePerc,:]
TP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
FP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']

print("Number of True Positive samples {}".format(TP.shape[0]))
print("Number of False Positive samples {}".format(FP.shape[0]))
print("% Actives {}".format(float(TP.shape[0]) / float(FP.shape[0])))
TP['PUBCHEM_CID'].to_csv('TPs.txt', index=False)
FP['PUBCHEM_CID'].to_csv('FPs.txt', index=False)
Actives_gcn = round((float(TP.shape[0]) / (float(FP.shape[0])+float(TP.shape[0])))*100,2)
print(Actives_gcn)

## Docking then GCN

In [ ]:
### NEW
# Sort values according to gcn predictions (top-rank of preds = 1)
import Scoring 
auc_final = []
threshold = []
for i in np.arange(0.05,0.95,0.05):
  thr = i
  threshold.append(i)
  preds_active = df_gcn[df_gcn['preds'] >= thr]   # Change values
  preds_active = preds_active.sort_values(by='SCORE.INTER', ascending=True)
  preds_inactive = df_gcn[df_gcn['preds'] < thr]
  preds_inactive = preds_inactive.sort_values(by='SCORE.INTER', ascending=True)
  preds_active = preds_active.append(preds_inactive)
### Load Scoring module (from RDkit) and compute AUC (ROC) and Enrichment curve 
# Note that we have not load the RDkit packeage but only the Scoring.py which is in the current (local) working directory 

  y = np.array(preds_active['PUBCHEM_ACTIVITY_OUTCOME'])
  scores = np.array(-preds_active['SCORE.INTER'])
  y_asToF = (y=='Active') #como verdadero o falso
  scores2 = np.array([scores,y_asToF]).T
  auc = Scoring.CalcAUC(scores2, 1)
  auc_final.append(auc)


<ipython-input-39-88e59dc7b85e>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds_active = preds_active.append(preds_inactive)
<ipython-input-39-88e59dc7b85e>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds_active = preds_active.append(preds_inactive)
<ipython-input-39-88e59dc7b85e>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds_active = preds_active.append(preds_inactive)
<ipython-input-39-88e59dc7b85e>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds_active = preds_active.append(preds_inactive)
<ipython-input-39-88e59dc7b85e>:13: FutureWarning: The frame.append method is deprecated and will be removed fro

In [ ]:
df_auc = pd.DataFrame({'auc': auc_final, 'thr': threshold})
print(df_auc)

         auc   thr
0   0.625809  0.05
1   0.719450  0.10
2   0.749537  0.15
3   0.758573  0.20
4   0.771117  0.25
5   0.767513  0.30
6   0.775458  0.35
7   0.775775  0.40
8   0.781611  0.45
9   0.785562  0.50
10  0.785024  0.55
11  0.784512  0.60
12  0.774992  0.65
13  0.745624  0.70
14  0.723128  0.75
15  0.673803  0.80
16  0.624687  0.85
17  0.622727  0.90


In [ ]:
### NEW
# Sort values according to gcn predictions (top-rank of preds = 1)
thr = 0.5
preds_active = df_gcn[df_gcn['preds'] >= thr]   # Change values
preds_active = preds_active.sort_values(by='SCORE.INTER', ascending=True)
preds_inactive = df_gcn[df_gcn['preds'] < thr]
preds_inactive = preds_inactive.sort_values(by='SCORE.INTER', ascending=True)
preds_active = preds_active.append(preds_inactive)

<ipython-input-41-585707cfb20c>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds_active = preds_active.append(preds_inactive)


In [ ]:
preds_active.head(15)

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,preds,labels,PUBCHEM_ACTIVITY_OUTCOME
14810,71998,135474672,135474672,-33.986,-27.089,0.000,0.592225,0.0,Inactive
7,20,83511,83511,-31.440,-26.637,0.000,0.875673,1.0,Active
14508,70576,135403579,135403579,-28.469,-25.884,0.000,0.867101,0.0,Inactive
14808,71981,135473476,135473476,-27.089,-25.863,0.020,0.738096,0.0,Inactive
14717,71580,135449241,135449241,-25.358,-25.591,0.000,0.777555,0.0,Inactive
6638,31537,2330332,2330332,-31.805,-25.547,0.000,0.776067,0.0,Inactive
2271,10420,663899,663899,-24.493,-25.265,0.000,0.712315,0.0,Inactive
222,607,5390001,5390001,-25.866,-25.222,0.000,0.772297,1.0,Active
14187,69094,16034798,16034798,-29.392,-25.162,0.000,0.620446,0.0,Inactive
7164,34480,2837680,2837680,-27.359,-24.655,0.000,0.780375,0.0,Inactive


In [ ]:
### Load Scoring module (from RDkit) and compute AUC (ROC) and Enrichment curve 
# Note that we have not load the RDkit packeage but only the Scoring.py which is in the current (local) working directory 
import Scoring 
y_GCN_docking = np.array(preds_active['PUBCHEM_ACTIVITY_OUTCOME'])
scores_GCN_docking = np.array(-preds_active['SCORE.INTER'])
y_asToF = (y_GCN_docking=='Active') #como verdadero o falso
scores_GCN_docking_2 = np.array([scores_GCN_docking,y_asToF]).T
auc = Scoring.CalcAUC(scores_GCN_docking_2, 1)
auc_dockign_gcn = round(auc, 3)
print(auc_dockign_gcn)

0.786


In [ ]:
#Compute enrichments 

#fractions = np.arange(0.0005,1.0, 0.01)
#Scoring.CalcEnrichment(scores=scores2, col=1, fractions=fractions)

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve = Scoring.CalcPorc(scores_GCN_docking_2, 1, fractions) #curve = [porcActives, porcTotal]


# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_GCN_docking,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve[1], curve[0])
plt.legend(['random', 'enrichment'])
plt.title('Docking priority + GCN filter enrichment')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

plt.savefig(data_dir+"DOCKING_GCN_merge_enrichment_"+str(thr)+".png") #to save
plt.close()

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve = Scoring.CalcPorc(scores2, 1, fractions) #curve = [porcActives, porcTotal]


# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve[1], curve[0])
plt.ylim(0, 40)
plt.xlim(0, 40)
plt.legend(['random', 'enrichment'])
plt.title('Docking priority + GCN filter enrichment')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

#plt.savefig(data_dir+"DOCKING_GCN_merge_enrichment_zoom40-40.png") #to save
plt.close()

In [ ]:
# Get the molecules on the first 1% (then narrow down to 0.3%) of database 

import math
numMol = len(scores_GCN_docking)
numPerFrac = [math.ceil(numMol*f) for f in fractions]
onePerc = np.where(fractions >= .03)[0][0]
numMol_onePerc = int(numPerFrac[onePerc]) # number of molecules in the first 0.3 percent of database 
numMol_onePerc

In [ ]:
# Print out True Positives and False Positives from the top ranked 0.3% of database
df_onePerc = preds_active.iloc[:numMol_onePerc,:]
TP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
FP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']

print("Number of True Positive samples {}".format(TP.shape[0]))
print("Number of False Positive samples {}".format(FP.shape[0]))
print("% Actives {}".format(float(TP.shape[0]) / float(FP.shape[0])))
TP['PUBCHEM_CID'].to_csv('TPs.txt', index=False)
FP['PUBCHEM_CID'].to_csv('FPs.txt', index=False)
Actives_gcn_docking = round((float(TP.shape[0]) / (float(FP.shape[0])+float(TP.shape[0])))*100,2)
print(Actives_gcn_docking)

## GCN then DOCKING + PF4

In [ ]:
df_gcn.head()

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,preds,labels,PUBCHEM_ACTIVITY_OUTCOME
11771,57177,6399289,6399289,-17.960,-15.017,1.388,0.898365,0.0,Inactive
11896,57761,6604293,6604293,-25.021,-7.502,0.000,0.895749,0.0,Inactive
12503,60628,9555971,9555971,-16.191,-14.602,0.000,0.886688,0.0,Inactive
180,502,3856851,3856851,-25.805,-18.929,0.000,0.886666,1.0,Active
11907,57825,6604396,6604396,-20.697,-14.239,0.000,0.886413,0.0,Inactive


In [ ]:
### NEW
# Sort values according to gcn predictions (top-rank of preds = 1)
thr = 0.5
preds_active = df_gcn[df_gcn['preds'] >= thr]   # Change values
preds_active = preds_active.sort_values(by='SCORE.INTER', ascending=True)
preds_inactive = df_gcn[df_gcn['preds'] < thr]
preds_inactive = preds_inactive.sort_values(by='SCORE.INTER', ascending=True)
preds_active = preds_active.append(preds_inactive)

<ipython-input-393-585707cfb20c>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds_active = preds_active.append(preds_inactive)


In [ ]:
preds_active

,REC,_TITLE1,PUBCHEM_CID,SCORE,SCORE.INTER,SCORE.RESTR,preds,labels,PUBCHEM_ACTIVITY_OUTCOME
14810,71998,135474672,135474672,-33.986,-27.089,0.000,0.592225,0.0,Inactive
7,20,83511,83511,-31.440,-26.637,0.000,0.875673,1.0,Active
14508,70576,135403579,135403579,-28.469,-25.884,0.000,0.867101,0.0,Inactive
14808,71981,135473476,135473476,-27.089,-25.863,0.020,0.738096,0.0,Inactive
14717,71580,135449241,135449241,-25.358,-25.591,0.000,0.777555,0.0,Inactive
...,...,...,...,...,...,...,...,...,...
2065,9378,660141,660141,-10.479,-2.306,0.000,0.050535,0.0,Inactive
7112,34151,2827330,2827330,-3.803,-1.954,0.006,0.057409,0.0,Inactive
8234,39729,3015189,3015189,-6.308,-1.926,0.000,0.151484,0.0,Inactive
549,1313,8210,8210,-3.735,-1.819,0.000,0.050910,0.0,Inactive


In [ ]:
### Load Scoring module (from RDkit) and compute AUC (ROC) and Enrichment curve 
# Note that we have not load the RDkit packeage but only the Scoring.py which is in the current (local) working directory 
import Scoring 
y_GCN_docking = np.array(preds_active['PUBCHEM_ACTIVITY_OUTCOME'])
scores_GCN_docking = np.array(-preds_active['SCORE.INTER'])
y_asToF = (y_GCN_docking=='Active') #como verdadero o falso
scores_GCN_docking_2 = np.array([scores_GCN_docking,y_asToF]).T
auc = Scoring.CalcAUC(scores_GCN_docking_2, 1)
auc_dockign_gcn = round(auc, 3)
print(auc_dockign_gcn)

0.786


# **Mixing plots**

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve_docking = Scoring.CalcPorc(scores_docking2, 1, fractions) #curve without Pfs
curve_PFs = Scoring.CalcPorc(scores_PF4_2, 1, fractions) # Docking using PFs
curve_GCN_preds = Scoring.CalcPorc(scores_GCN_2, 1, fractions) # sort by GCN preds
curve_GCN_docking = Scoring.CalcPorc(scores_GCN_docking_2, 1, fractions) # GCN then docking

# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_GCN,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)
auc = Scoring.CalcAUC(scores3, 1)
auc = round(auc, 3)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve_docking[1], curve_docking[0])
plt.plot(curve_PFs[1], curve_PFs[0])
plt.plot(curve_GCN_preds[1], curve_GCN_preds[0])
plt.plot(curve_GCN_docking[1], curve_GCN_docking[0])
plt.legend(['random AUC ='+str(auc), 'docking AUC ='+str(auc_docking),'PFs AUC ='+str(auc_PFs),'GCN AUC ='+str(auc_GCN),'GCN+docking AUC ='+str(auc_dockign_gcn)])
plt.title('Enrichment comparative plots')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

plt.savefig(data_dir+"mix_docking_gcn_4.png") #to save
plt.close()

## Without AUC

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve_docking = Scoring.CalcPorc(scores_docking2, 1, fractions) #curve without Pfs
curve_PFs = Scoring.CalcPorc(scores_PF4_2, 1, fractions) # Docking using PFs
curve_GCN_preds = Scoring.CalcPorc(scores_GCN_2, 1, fractions) # sort by GCN preds
curve_GCN_docking = Scoring.CalcPorc(scores_GCN_docking_2, 1, fractions) # GCN then docking

# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_GCN,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)
auc = Scoring.CalcAUC(scores3, 1)
auc = round(auc, 3)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve_docking[1], curve_docking[0])
plt.plot(curve_PFs[1], curve_PFs[0])
plt.plot(curve_GCN_preds[1], curve_GCN_preds[0])
plt.plot(curve_GCN_docking[1], curve_GCN_docking[0])
plt.legend(['random','docking','PFs','GCN','GCN+Docking'])
plt.title('Enrichment comparative plots')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

plt.savefig(data_dir+"mix_docking_gcn_4_WTH.png") #to save
plt.close()

In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve_docking = Scoring.CalcPorc(scores_docking2, 1, fractions) #curve without Pfs
curve_PFs = Scoring.CalcPorc(scores_PF4_2, 1, fractions) # Docking using PFs
curve_GCN_preds = Scoring.CalcPorc(scores_GCN_2, 1, fractions) # sort by GCN preds
curve_GCN_docking = Scoring.CalcPorc(scores_GCN_docking_2, 1, fractions) # GCN then docking

# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_docking,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)
auc = Scoring.CalcAUC(scores3, 1)
auc = round(auc, 3)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve_docking[1], curve_docking[0])
plt.plot(curve_PFs[1], curve_PFs[0])
plt.plot(curve_GCN_preds[1], curve_GCN_preds[0])
plt.plot(curve_GCN_docking[1], curve_GCN_docking[0])
plt.ylim(0, 20)
plt.xlim(0, 20)
plt.legend(['random AUC ='+str(auc), 'docking AUC ='+str(auc_docking),'PFs AUC ='+str(auc_PFs),'GCN AUC ='+str(auc_GCN),'GCN+docking AUC ='+str(auc_dockign_gcn)])
plt.title('Enrichment comparative plots')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

plt.savefig(data_dir+"mix_docking_gcn_zoom20-20_3.png") #to save
plt.close()

<ipython-input-45-d87f0f150169>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  scores3 = np.array([scores_docking,y_as_bin]).T


In [ ]:
# compute and plot the percentajes of actives versus percentaje of database like in DUD paper 
curve_docking = Scoring.CalcPorc(scores_docking2, 1, fractions) #curve without Pfs
curve_PFs = Scoring.CalcPorc(scores_PF4_2, 1, fractions) # Docking using PFs
curve_GCN_preds = Scoring.CalcPorc(scores_GCN_2, 1, fractions) # sort by GCN preds
curve_GCN_docking = Scoring.CalcPorc(scores_GCN_docking_2, 1, fractions) # GCN then docking

# Shuffle the inhibitory outcomes from array (y_asToF) so that it looks randomly sorted. Then compute the enrichment from the random array
y_as_bin = y_asToF
np.random.shuffle(y_as_bin)
scores3 = np.array([scores_GCN,y_as_bin]).T
curve_random = Scoring.CalcPorc(scores3, 1, fractions)
auc = Scoring.CalcAUC(scores3, 1)
auc = round(auc, 3)

import matplotlib.pyplot as plt 
%matplotlib inline 
plt.figure(figsize=(8, 6))
plt.plot(curve_random[1], curve_random[0])
plt.plot(curve_docking[1], curve_docking[0])
plt.plot(curve_PFs[1], curve_PFs[0])
plt.plot(curve_GCN_preds[1], curve_GCN_preds[0])
plt.plot(curve_GCN_docking[1], curve_GCN_docking[0])
plt.ylim(0, 20)
plt.xlim(0, 20)
plt.legend(['random','docking','PFs','GCN','GCN+Docking'])
plt.title('Enrichment comparative plots')
plt.xlabel('% of Database')
plt.ylabel('% of Actives')

plt.savefig(data_dir+"mix_docking_gcn_zoom20-20_4_WTH.png") #to save
plt.close()

In [ ]:
models = "docking","docking+PFs","GCN","GCN+Docking"
top3 = Actives_docking,Actives_PF4, Actives_gcn, Actives_gcn_docking


In [ ]:
# Get the molecules on the first 1% (then narrow down to 0.3%) of database 

import math
numMol = len(scores_GCN_docking)
numPerFrac = [math.ceil(numMol*f) for f in fractions]
onePerc = np.where(fractions >= .03)[0][0]
numMol_onePerc = int(numPerFrac[onePerc]) # number of molecules in the first 0.3 percent of database 
numMol_onePerc # SAme for all

In [ ]:
datasets_auc = df, VS_table_with_labels_PFs, df_gcn, preds_active

In [ ]:
df.shape[0]

In [ ]:
import pandas as pd
import numpy as np
import math

datasets_auc = {'Dockings': df, 'Dockings_PFs': VS_table_with_labels_PFs, 'GCN': df_gcn, 'GCN+Dockings': preds_active}
thr = [0.005,0.01,0.02,0.03,0.05,0.1,0.15,0.2,0.25,0.5,0.7,0.99]
fractions = np.arange(0.0005,1.0, 0.01)

df_final = pd.DataFrame()

for name, data in datasets_auc.items():
    results = [] # Lista vacía para almacenar los resultados
    for j in thr:
       # print("Dataset: {}, fraction %{}".format(name, j*100))
        numMol = data.shape[0]
        numPerFrac = [math.ceil(numMol*f) for f in fractions]
        try:
            onePerc = np.where(fractions >= j)[0][0]
            numMol_onePerc = int(numPerFrac[onePerc])
            df_onePerc = data.iloc[:numMol_onePerc,:]
            TP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
            FP = df_onePerc[df_onePerc['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']
            Actives_gcn_docking = round((float(TP.shape[0]) / (float(FP.shape[0])+float(TP.shape[0])))*100,2)
            #print(Actives_gcn_docking)
            results.append({'% Top Dataset': j*100, name: Actives_gcn_docking})
        except:
            pass

# Crear un DataFrame de pandas con los resultados
    df_results = pd.DataFrame(results)
    df_final = pd.concat([df_final,df_results],axis=1)

# Mostrar el DataFrame
df_final =df_final.iloc[:,[0,1,3,5,7]]
df_final

,% Top Dataset,Dockings,Dockings_PFs,GCN,GCN+Dockings
0,0.5,3.36,5.68,10.19,8.28
1,1.0,3.36,5.68,10.19,8.28
2,2.0,2.85,4.97,8.82,9.48
3,3.0,2.71,4.92,8.77,9.67
4,5.0,2.63,4.72,9.67,8.89
5,10.0,2.43,3.77,8.58,7.72
6,15.0,2.20,3.29,7.60,6.53
7,20.0,1.93,2.85,6.40,6.30
8,25.0,1.80,2.54,5.60,5.63
9,50.0,1.44,1.77,3.50,3.34


In [ ]:
max_value = df_final.iloc[:,[1,2,3,4]].max().max()
print(max_value)

In [ ]:
import pandas as pd

# Creamos un diccionario con los datos
data = {
    'Models': ["docking", "docking+PFs", "GCN", "GCN+Docking"],
    '% Activos del Top3': [Actives_docking, Actives_PF4, Actives_gcn, Actives_gcn_docking],
}

# Creamos el DataFrame
df_actives = pd.DataFrame(data)

# Imprimimos el DataFrame
df_actives